# Scratchpad Agents
AI Agents sometimes forget, mistakenly summarise content and incorrectly pass information on. One solution to solve this problem is to provide agents with the capability to store and retrieve information.

In [1]:
import time
from openai import OpenAI
from agents import BDRAgent, LocalBDRAgent, RedisBDRAgent, StaticBDRAgent
from tool_schema import tools
client = OpenAI()

USER_INPUT = """Can you write a personalised outreach email for Michael Zhao? 
Use their profile information and reference relevant value propositions.
Use information from the email thread content to make the email more personalised."""

## Status Quo: Agents without scratchpads
AI agents currently regenerate and pass on information to subsequent sub-agents or tools through function call arguments. This can lead to fragile results or hallucinations. We see below that once the agent is provided with a series of complex tasks, it begins to incorrectly retrieve past information.

In [2]:
start_time = time.perf_counter()
agent = BDRAgent(tools=tools, client=client)
agent.respond_to_user_input(USER_INPUT)
print("--- %s seconds ---" % ("{:.2f}".format(time.perf_counter() - start_time)))

Function generation requested, calling function extract_profile_information...
Getting profile information for Michael Zhao...
Function generation requested, calling function retrieve_relevant_value_prop...
Retrieving relevant value propositions...
Function generation requested, calling function get_email_thread_id...
Getting email thread ID for Michael Zhao...
Function generation requested, calling function retrieve_email_thread_content...
Retrieving email thread content using A0L31ZE5YS...
--- [CHECK] Is the email thread ID the same? True ---
Function generation requested, calling function compose_outreach_email...
Composing outreach email...
--- [CHECK] Is the email thread ID the same? True ---
--- [CHECK] Has the previous email thread content been maintained? True ---
--- [CHECK] Has the full value proposition been maintained? False ---
Function generation requested, calling function compose_outreach_email...
Composing outreach email...
--- [CHECK] Is the email thread ID the same? 

In [3]:
agent.conversation.display_conversation()

system: You're a world class BDR researcher who can do detailed research on any topic and produce facts-based results. 
You work for Emily who is a account executive at EdApp. When responded to external messages reply as if you are Emily. 

Please make sure you complete the objective following these rules:
1/ For each research task, you should try to find any recent news about the prospect company, or recent posts.
2/ When using google search, use quotation mark to wrap around the company name, e.g. "Salesforce", because some company name is generic, this help google return better results; 
3/ You should do enough research to gather as much information as possible about the objective. If there are URL's of relevant links & articles, you will scrape it to gather more information. After scraping & search, you should think "are there any new things I should search & scrape based on the data I collected to increase the research quality?" If the answer is yes: continue. But don't do this mo

## Proposed Solution #0: Leave the Agents alone! Why not update the tools instead?
Perhaps the most intuitive method of ensuring that agents do not mess with your data is to keep it out of their hands entirely. We can do this by creating short term caches for all tool and function calls, and ensuring that functions or tools call each other when they are dependent on each other. However, the obvious and non-start reason is that this approach takes away from the magic of AI agents. Functions would be chained together like traditional computing. We would also need to individually update each tool. 

In [4]:
from tool_schema_static import tools_static
start_time = time.perf_counter()
agent = StaticBDRAgent(tools=tools_static, client=client)
agent.respond_to_user_input(USER_INPUT)
print("--- %s seconds ---" % ("{:.2f}".format(time.perf_counter() - start_time)))

Function generation requested, calling function extract_profile_information...
Getting profile information for Michael Zhao...
Function generation requested, calling function retrieve_relevant_value_prop...
Retrieving relevant value propositions for Michael Zhao...
Function generation requested, calling function get_email_thread_id...
Getting email thread ID for Michael Zhao...
Function generation requested, calling function retrieve_email_thread_content...
Retrieving email thread content using LH86BEWCB5...
--- [CHECK] Is the email thread ID the same? True ---
Function generation requested, calling function compose_outreach_email...
Composing outreach email...
--- [CHECK] Is the email thread ID the same? True ---
--- [CHECK] Has the previous email thread content been maintained? True ---
--- [CHECK] Has the full value proposition been maintained? True ---
Function generation requested, calling function compose_outreach_email...
Function not required, responding to user
--- 28.22 secon

In [5]:
agent.conversation.display_conversation()

system: You're a world class BDR researcher who can do detailed research on any topic and produce facts-based results. 
You work for Emily who is a account executive at EdApp. When responded to external messages reply as if you are Emily. 

Please make sure you complete the objective following these rules:
1/ For each research task, you should try to find any recent news about the prospect company, or recent posts.
2/ When using google search, use quotation mark to wrap around the company name, e.g. "Salesforce", because some company name is generic, this help google return better results; 
3/ You should do enough research to gather as much information as possible about the objective. If there are URL's of relevant links & articles, you will scrape it to gather more information. After scraping & search, you should think "are there any new things I should search & scrape based on the data I collected to increase the research quality?" If the answer is yes: continue. But don't do this mo

*Running it again to see performance improvement from storing outputs*

In [6]:
from tool_schema_static import tools_static
start_time = time.perf_counter()
agent = StaticBDRAgent(tools=tools_static, client=client)
agent.respond_to_user_input(USER_INPUT)
print("--- %s seconds ---" % ("{:.2f}".format(time.perf_counter() - start_time)))

Function generation requested, calling function extract_profile_information...
Function generation requested, calling function retrieve_relevant_value_prop...
Function generation requested, calling function get_email_thread_id...
Function generation requested, calling function retrieve_email_thread_content...
Function generation requested, calling function compose_outreach_email...
Function generation requested, calling function compose_outreach_email...
Function not required, responding to user
--- 10.71 seconds ---


## Proposed Solution #1: Agent with local, in-memory scratchpad
Local, in-memory storage provides fast, reliable and efficient storage of short-term information. It is also easy to implement. We see that even with this simple implementation, inputs from previoius function calls are perfectly passed along. Responses are accurate and fast.

In [7]:
start_time = time.perf_counter()
local_storage = {}
agent = LocalBDRAgent(tools=tools, client=client, scratchpad=local_storage)
agent.respond_to_user_input(USER_INPUT)
print("--- %s seconds ---" % ("{:.2f}".format(time.perf_counter() - start_time)))

Function generation requested, calling function extract_profile_information...
Getting profile information for Michael Zhao...
Function generation requested, calling function retrieve_relevant_value_prop...
Retrieving relevant value propositions...
Function generation requested, calling function get_email_thread_id...
Getting email thread ID for Michael Zhao...
Function generation requested, calling function retrieve_email_thread_content...
Retrieving email thread content using A0L31ZE5YS...
--- [CHECK] Is the email thread ID the same? True ---
Function generation requested, calling function compose_outreach_email...
Composing outreach email...
--- [CHECK] Is the email thread ID the same? True ---
--- [CHECK] Has the previous email thread content been maintained? True ---
--- [CHECK] Has the full value proposition been maintained? True ---
Function generation requested, calling function compose_outreach_email...
Composing outreach email...
--- [CHECK] Is the email thread ID the same? T

In [8]:
agent.conversation.display_conversation()

system: You're a world class BDR researcher who can do detailed research on any topic and produce facts-based results. 
You work for Emily who is a account executive at EdApp. When responded to external messages reply as if you are Emily. 

Please make sure you complete the objective following these rules:
1/ For each research task, you should try to find any recent news about the prospect company, or recent posts.
2/ When using google search, use quotation mark to wrap around the company name, e.g. "Salesforce", because some company name is generic, this help google return better results; 
3/ You should do enough research to gather as much information as possible about the objective. If there are URL's of relevant links & articles, you will scrape it to gather more information. After scraping & search, you should think "are there any new things I should search & scrape based on the data I collected to increase the research quality?" If the answer is yes: continue. But don't do this mo

## Proposed Solution #2: Agent with remote scratchpad
Agents often work in teams, and may not necessarily share the same internal memory. Remote data stores are promising solutions that offer scalability, robustness and sharability. In this implementation, we use Redis.

In [9]:
!redis-server

8237:C 26 Apr 2024 14:33:52.259 * oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
8237:C 26 Apr 2024 14:33:52.259 * Redis version=7.2.4, bits=64, commit=00000000, modified=0, pid=8237, just started
8237:C 26 Apr 2024 14:33:52.259 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
8237:M 26 Apr 2024 14:33:52.259 * monotonic clock: POSIX clock_gettime
                _._                                                  
           _.-``__ ''-._                                             
      _.-``    `.  `_.  ''-._           Redis 7.2.4 (00000000/0) 64 bit
  .-`` .-```.  ```\/    _.,_ ''-._                                  
 (    '      ,       .-`  | `,    )     Running in standalone mode
 |`-._`-...-` __...-.``-._|'` _.-'|     Port: 6379
 |    `-._   `._    /     _.-'    |     PID: 8237
  `-._    `-._  `-./  _.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|             

In [10]:
import redis

start_time = time.perf_counter()
redis_cache = redis.Redis(host='localhost', port=6379, decode_responses=True)
agent = RedisBDRAgent(tools=tools, client=client, scratchpad=redis_cache)
agent.respond_to_user_input(USER_INPUT)
print("--- %s seconds ---" % ("{:.2f}".format(time.perf_counter() - start_time)))

Function generation requested, calling function extract_profile_information...
Getting profile information for Michael Zhao...
Function generation requested, calling function retrieve_relevant_value_prop...
Retrieving relevant value propositions...
Function generation requested, calling function get_email_thread_id...
Getting email thread ID for Michael Zhao...
Function generation requested, calling function retrieve_email_thread_content...
Retrieving email thread content using A0L31ZE5YS...
--- [CHECK] Is the email thread ID the same? True ---
Function generation requested, calling function compose_outreach_email...
Composing outreach email...
--- [CHECK] Is the email thread ID the same? True ---
--- [CHECK] Has the previous email thread content been maintained? True ---
--- [CHECK] Has the full value proposition been maintained? True ---
Function generation requested, calling function compose_outreach_email...
Composing outreach email...
--- [CHECK] Is the email thread ID the same? T

In [11]:
agent.conversation.display_conversation()

system: You're a world class BDR researcher who can do detailed research on any topic and produce facts-based results. 
You work for Emily who is a account executive at EdApp. When responded to external messages reply as if you are Emily. 

Please make sure you complete the objective following these rules:
1/ For each research task, you should try to find any recent news about the prospect company, or recent posts.
2/ When using google search, use quotation mark to wrap around the company name, e.g. "Salesforce", because some company name is generic, this help google return better results; 
3/ You should do enough research to gather as much information as possible about the objective. If there are URL's of relevant links & articles, you will scrape it to gather more information. After scraping & search, you should think "are there any new things I should search & scrape based on the data I collected to increase the research quality?" If the answer is yes: continue. But don't do this mo

In [12]:
!redis-cli FLUSHDB

OK
